<h1>Capstone project comparing Hong Kong and Singapore -HK

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h2>Define Foursquare Credentials and Version

<h2>Geo Coordinates of Hong Kong 

In [3]:
address = 'Central District, Hong Kong, China'

geolocator = Nominatim(user_agent="hk_explorer")
location = geolocator.geocode(address)
hklatitude = location.latitude
hklongitude = location.longitude
print('The geograpical coordinate of Central, Hong Kong are {}, {}.'.format(hklatitude, hklongitude))

The geograpical coordinate of Central, Hong Kong are 22.2813223, 114.1602579.


<H2> Map of Hong Kong 

In [4]:
hk_map = folium.Map(location=[hklatitude, hklongitude], zoom_start=15)
hk_map
folium.features.CircleMarker(
    [hklatitude, hklongitude],
    radius=30,
    color='red',
    popup='Downtown',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hk_map)
hk_map

<h2>HK Foursquare Analysis

In [6]:
results = requests.get(url).json()

In [7]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
hkvenues = results['response']['groups'][0]['items']
    
hknearby_venues = json_normalize(hkvenues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
hknearby_venues =hknearby_venues.loc[:, filtered_columns]

# filter the category for each row
hknearby_venues['venue.categories'] = hknearby_venues.apply(get_category_type, axis=1)

# clean columns
hknearby_venues.columns = [col.split(".")[-1] for col in hknearby_venues.columns]

hknearby_venues.head(5)

,name,categories,lat,lng
0,Mandarin Grill + Bar (Mandarin Grill + Bar 文華扒...,Steakhouse,22.281462,114.160156
1,Mott 32 (卅二公館),Dim Sum Restaurant,22.280696,114.159380
2,Mandarin Oriental Hong Kong (香港文華東方酒店),Hotel,22.281879,114.159443
3,8½ Otto e Mezzo Bombana,Italian Restaurant,22.281726,114.158767
4,The Mandarin Cake Shop,Bakery,22.281959,114.159416


In [9]:
hknearby_venues.sort_values('categories', ascending=True).head(5)

,name,categories,lat,lng
79,In-Town Check-In (市區預辦登機),Airport Service,22.284973,114.158306
45,Gagosian Gallery,Art Gallery,22.281805,114.157140
88,Brass Spoon,Asian Restaurant,22.283329,114.156093
64,Grill ขุนทอง,BBQ Joint,22.281279,114.155596
4,The Mandarin Cake Shop,Bakery,22.281959,114.159416


In [10]:
print('{} venues were returned by Foursquare.'.format(hknearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [11]:
# create map of Central using latitude and longitude values
map_central = folium.Map(location=[hklatitude, hklongitude], zoom_start=16)

# add markers to map
for lat, lng, label in zip(hknearby_venues['lat'], hknearby_venues['lng'], hknearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central)  
    
map_central

<h2>Ranking the Catagories of venues 

In [12]:
hkgroup=hknearby_venues.groupby('categories').count()
hkgroup.head(5)

,name,lat,lng
categories,,,
Airport Service,1,1,1
Art Gallery,1,1,1
Asian Restaurant,1,1,1
BBQ Joint,1,1,1
Bakery,1,1,1


In [13]:
hkgroup.sort_values('name', ascending=False).head(5)

,name,lat,lng
categories,,,
Chinese Restaurant,5,5,5
Lounge,5,5,5
Italian Restaurant,4,4,4
Sushi Restaurant,4,4,4
Gym / Fitness Center,4,4,4


In [15]:
# selecet restaurarnt from categories 
# mark coffee shops on the map 
# analysis of Starbucks' presence in Central, Hong Kong and Downtown Singapore

In [16]:
hkgroup2=hkgroup.filter(like='Restaurant', axis=0)  
hkgroup2.sort_values('name', ascending=False)
hkgroup2.drop(['lat','lng'], axis=1, inplace=True)
hkgroup2.sort_values('name', ascending=False)
hkgroup2.rename(columns={'name': 'counts'}, inplace=True)
hkgroup2.sort_values('counts', ascending=False)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,counts
categories,
Chinese Restaurant,5
French Restaurant,4
Italian Restaurant,4
Sushi Restaurant,4
Cantonese Restaurant,3
Japanese Restaurant,3
Dim Sum Restaurant,2
Asian Restaurant,1
Beijing Restaurant,1


<h2>Locating Starbucks around the city centre

In [18]:
search_query = 'Starbucks'
radius = 500
print(search_query + ' .... OK!')

Starbucks .... OK!


In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc958784434b9319ee43602'},
 'response': {'venues': [{'id': '4cb50d2664998cfac16a0ba2',
    'name': 'Starbucks (星巴克)',
    'location': {'address': 'Kiosk CEN K4-K5, Transfer Concourse, MTR Central Station',
     'lat': 22.281046646302286,
     'lng': 114.15977020776883,
     'labeledLatLngs': [{'label': 'display',
       'lat': 22.281046646302286,
       'lng': 114.15977020776883}],
     'distance': 58,
     'cc': 'HK',
     'city': '中環',
     'state': '中西區',
     'country': '香港',
     'formattedAddress': ['Kiosk CEN K4-K5, Transfer Concourse, MTR Central Station',
      '中西區',
      '香港']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1556699256',
    'hasPerk': False},
   {'id': '4b05e882

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4cb50d2664998cfac16a0ba2,"Kiosk CEN K4-K5, Transfer Concourse, MTR Centr...",HK,中環,香港,NaN,58,"[Kiosk CEN K4-K5, Transfer Concourse, MTR Cent...","[{'label': 'display', 'lat': 22.28104664630228...",22.281047,114.159770,NaN,中西區,Starbucks (星巴克),v-1556699256
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b05e882f964a52096e522e3,"Shop 201, 2/F, Alexandra House, 16-20 Chater Rd",HK,中環,香港,NaN,187,"[Shop 201, 2/F, Alexandra House, 16-20 Chater ...","[{'label': 'display', 'lat': 22.28166667371219...",22.281667,114.158478,Central,中西區,Starbucks (星巴克),v-1556699256
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b13a67ff964a520379823e3,"Shop M2, M/F, Baskerville House, 13 Duddell St",HK,中環,香港,NaN,378,"[Shop M2, M/F, Baskerville House, 13 Duddell S...","[{'label': 'display', 'lat': 22.27982617005152...",22.279826,114.156959,Central,中西區,Starbucks (星巴克),v-1556699256
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4b3aa180f964a5208a6b25e3,"G/F, Jardine House, 1 Connaught Pl",HK,中環,香港,NaN,249,"[G/F, Jardine House, 1 Connaught Pl, 中西區, 香港]","[{'label': 'display', 'lat': 22.28316087787933...",22.283161,114.158879,NaN,中西區,Starbucks (星巴克),v-1556699256
4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4f78279ae4b0ca8cfda079b8,"1/F, Prosperity Tower, 39 Queen's Rd C",HK,中環,香港,NaN,463,"[1/F, Prosperity Tower, 39 Queen's Rd C, 中西區, 香港]","[{'label': 'display', 'lat': 22.28257267762816...",22.282573,114.155962,Central,中西區,Starbucks (星巴克),v-1556699256


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,state,id
0,Starbucks (星巴克),Coffee Shop,"Kiosk CEN K4-K5, Transfer Concourse, MTR Centr...",HK,中環,香港,NaN,58,"[Kiosk CEN K4-K5, Transfer Concourse, MTR Cent...","[{'label': 'display', 'lat': 22.28104664630228...",22.281047,114.159770,NaN,中西區,4cb50d2664998cfac16a0ba2
1,Starbucks (星巴克),Coffee Shop,"Shop 201, 2/F, Alexandra House, 16-20 Chater Rd",HK,中環,香港,NaN,187,"[Shop 201, 2/F, Alexandra House, 16-20 Chater ...","[{'label': 'display', 'lat': 22.28166667371219...",22.281667,114.158478,Central,中西區,4b05e882f964a52096e522e3
2,Starbucks (星巴克),Coffee Shop,"Shop M2, M/F, Baskerville House, 13 Duddell St",HK,中環,香港,NaN,378,"[Shop M2, M/F, Baskerville House, 13 Duddell S...","[{'label': 'display', 'lat': 22.27982617005152...",22.279826,114.156959,Central,中西區,4b13a67ff964a520379823e3
3,Starbucks (星巴克),Coffee Shop,"G/F, Jardine House, 1 Connaught Pl",HK,中環,香港,NaN,249,"[G/F, Jardine House, 1 Connaught Pl, 中西區, 香港]","[{'label': 'display', 'lat': 22.28316087787933...",22.283161,114.158879,NaN,中西區,4b3aa180f964a5208a6b25e3
4,Starbucks (星巴克),Coffee Shop,"1/F, Prosperity Tower, 39 Queen's Rd C",HK,中環,香港,NaN,463,"[1/F, Prosperity Tower, 39 Queen's Rd C, 中西區, 香港]","[{'label': 'display', 'lat': 22.28257267762816...",22.282573,114.155962,Central,中西區,4f78279ae4b0ca8cfda079b8
5,Starbucks (星巴克),Coffee Shop,"Shop 2097-2098, 2/F, ifc mall, 8 Finance St",HK,中環,香港,NaN,414,"[Shop 2097-2098, 2/F, ifc mall, 8 Finance St, ...","[{'label': 'display', 'lat': 22.28493759454223...",22.284938,114.159276,Central,中西區,4d04c3729b7f60fc1cab09ab
6,Starbucks (星巴克),Coffee Shop,"Shop 308-310, 3/F, Exchange Square Podium",HK,中環,香港,NaN,400,"[Shop 308-310, 3/F, Exchange Square Podium, 中西...","[{'label': 'display', 'lat': 22.28408892734929...",22.284089,114.157774,NaN,中西區,4b28354df964a520e69024e3
7,Starbucks (星巴克),Coffee Shop,"1/F, West Hall, Bank of China Tower, 1 Garden Rd",HK,中環,香港,NaN,205,"[1/F, West Hall, Bank of China Tower, 1 Garden...","[{'label': 'display', 'lat': 22.27975229958153...",22.279752,114.161299,NaN,中西區,4fd9a208e4b0318de7a8d2b3
8,Starbucks (星巴克),Coffee Shop,"Shop 131, 1/F, Pacific Place, 88 Queensway",HK,金鐘,香港,NaN,578,"[Shop 131, 1/F, Pacific Place, 88 Queensway, 中...","[{'label': 'display', 'lat': 22.27808404906994...",22.278084,114.164651,NaN,中西區,4b206e66f964a520333224e3
9,Starbucks (星巴克),Coffee Shop,"Shop 3, G/F, 3 Garden Rd",HK,中環,香港,NaN,253,"[Shop 3, G/F, 3 Garden Rd, 中西區, 香港]","[{'label': 'display', 'lat': 22.27907445710351...",22.279074,114.160678,NaN,中西區,4baee095f964a5200adf3be3


In [23]:
dataframe_filtered.name

0               Starbucks (星巴克)
1               Starbucks (星巴克)
2               Starbucks (星巴克)
3               Starbucks (星巴克)
4               Starbucks (星巴克)
5               Starbucks (星巴克)
6               Starbucks (星巴克)
7               Starbucks (星巴克)
8               Starbucks (星巴克)
9               Starbucks (星巴克)
10    Starbucks Reserve (星巴克臻選)
11              Starbucks (星巴克)
12              Starbucks (星巴克)
13              Starbucks (星巴克)
14              Starbucks (星巴克)
15              Starbucks (星巴克)
16                    Starbucks
17                Starbucks 星巴克
18                    Starbucks
Name: name, dtype: object

In [24]:
hkstarbucks_map = folium.Map(location=[hklatitude, hklongitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
#folium.features.CircleMarker(
#    [hklatitude, hklongitude],
#    radius=10,
#    color='red',
#    popup='Centrel',
#    fill = True,
#    fill_color = 'red',
#    fill_opacity = 0.6
#).add_to(hkstarbucks_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hkstarbucks_map)

hkstarbucks_map

<h2>Plotting Non-Starbucks around the city centre

In [25]:
hk_coffee=hknearby_venues.loc[hknearby_venues['categories'] == 'Coffee Shop']
hk_coffee
hkcoffee_nonsb=hk_coffee[hk_coffee.name != 'Starbucks (星巴克)']
hkcoffee_nonsb

,name,categories,lat,lng
43,NODI,Coffee Shop,22.282049,114.158598
50,Fuel Espresso,Coffee Shop,22.280628,114.157730


In [26]:
hkstarbucks_map = folium.Map(location=[hklatitude, hklongitude], zoom_start=13) # generate map centred around the Conrad Hotel

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hkstarbucks_map)

for lat, lng, label in zip(hkcoffee_nonsb.lat, hkcoffee_nonsb.lng, hkcoffee_nonsb.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(hkstarbucks_map)

hkstarbucks_map


In [27]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Coffee Shops</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:blue;opacity:0.7;'></span>Starbucks</li>
    <li><span style='background:red;opacity:0.7;'></span>Non-Starbucks</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

hkstarbucks_map.get_root().add_child(macro)